## Datasets
### https://archive.ics.uci.edu/ml/machine-learning-databases/glass/
### https://archive.ics.uci.edu/ml/datasets/Heart+failure+clinical+records
### https://archive.ics.uci.edu/ml/datasets/HCC+Survival

In [3]:
import numpy as np

dataset1 = np.loadtxt('datasets/glass.data', delimiter=',')
dataset2 = np.loadtxt('datasets/heart_failure_clinical_records_dataset.csv', delimiter=',', skiprows=1)
dataset3 = np.loadtxt('datasets/hcc-data.txt', delimiter=',') # Dados faltantes preenchidos com 0

In [12]:
X1, y1 = dataset1[:,1:-1], dataset1[:,-1]

print(f"{X1[0]}")


[1.52101e+00 1.36400e+01 4.49000e+00 1.10000e+00 7.17800e+01 6.00000e-02
 8.75000e+00 0.00000e+00 0.00000e+00]
